In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from dash import Dash, html, dcc, Output, Input
import dash_bootstrap_components as dbc
pd.options.display.float_format = '{:,.0f}'.format
import glob

Carga de los datos de venta

In [2]:
lista = lista = glob.glob("D:/Zaeli/Analise Cliente Produto Comercial/Consolidado/Recientes/*.csv")
ventas = []

In [3]:
for archivo in lista:
    data = pd.read_csv(archivo, header=0, sep = ';', parse_dates = ['Ano/Mes'], dtype = {'CodGer':str,'CODSAP':str ,'CodRepresentante':str, 'Cli&Rep':str})
    data = data.rename(columns={'Ano/Mes':'Fecha', 'CODSAP' : 'Material'})
    ventas.append(data)
ventas = pd.concat(ventas, ignore_index = True)

Actualizacion por inflacion

In [4]:
inflacion = pd.read_excel("D:/Zaeli/BI/Archivos Compartidos/Inflacion.xlsx", header=3, sheet_name = 'Hoja1')
inflacion = inflacion.rename(columns= {'DataKey':'ChaveData'})
ventasi = pd.merge(ventas, inflacion)
ventasi['ReceitaBrutaIn'] = ventasi.ReceitaBruta * (1 + ventasi.InfacaoAcum)
ventasi['Verbas'] = ventasi.Bonif + ventasi.DescConcedidos + ventasi.Impostos + ventasi.Frete + ventasi.Comis - ventasi.Devol - ventasi.Troca
ventasi['ReceitaLiquida'] = ventasi.ReceitaBruta - ventasi.Verbas
ventasi['ReceitaLiquidaIn'] = ventasi.ReceitaLiquida * (1 + ventasi.InfacaoAcum)
ventasi['MargenBruto'] = ventasi.ReceitaLiquida - ventasi.CPV
ventasi['MargenBrutoIn'] = ventasi.MargenBruto * (1 + ventasi.InfacaoAcum)
ventasi = ventasi.drop(['PesoBruto', 'ReceitaBruta', 'Bonif', 'DescConcedidos', 'Impostos', 'Comis', 'CPV', 'Devol', 'Troca',                                                                       'Inflacaomes','InfacaoAcum', 'Verbas', 'MargenBruto', 'ReceitaLiquida'], axis=1)
ventasi = ventasi.rename(columns={
    'ReceitaBrutaIn' : 'ReceitaBruta',
    'ReceitaLiquidaIn' : 'ReceitaLiquida',
    'MargenBrutoIn' : 'Margen'
})

Carga de los productos

In [5]:
lineas = pd.read_csv("D:/Zaeli/PrevisionDemanda/lineasProducto.csv", header=0, sep=';')
lineas = lineas.rename(columns={
    'COD SAP':'Material',
    'Linha':'Linea',
    'Produto':'Producto'
})
lineas['Material'] = lineas['Material'].astype(str)
ventasP = pd.merge(ventasi, lineas)
ventasP = ventasP.drop(['Cli&Rep', 'Tipo'], axis=1)

Carga del cadastro de clientes

In [6]:
ListadoClientes = pd.read_excel("D:/Zaeli/BI/Archivos Compartidos/Clientes.xlsx",sheet_name='Listado')
ListadoClientes = ListadoClientes.drop(['Cidade', 'CodEstado', 'Estado', 'CidadeEstado'], axis=1)
ventasC = pd.merge(ventasP, ListadoClientes)

Consolidacion por mes, linea y municipio

In [7]:
ventasM = ventasC.groupby([pd.Grouper(freq='M', key = 'Fecha'), 'Fecha' ,'CodigoMunicipio', 'Linea', 'Producto'], as_index=False).agg({
    'ReceitaBruta':'sum',
    'ReceitaLiquida' : 'sum',
    'PesoLiq' :'sum',
    'Volume' : 'sum',
    'Margen': 'sum'
})